In [446]:
LOADABLE_ROWS = 3
FILE_NAME = 'ec_instance.txt'

In [447]:
class Matrix:
    def __init__(self, data, rows, cols):
        self.data = data
        self.rows = rows
        self.cols = cols

In [448]:
def count_comment_lines(filename):
    with open(filename, 'r') as f:
        return sum(1 for line in f if line.startswith(";;;"))

In [449]:
def count_total_lines(filename):
    with open(filename, 'r') as f:
        return sum(1 for line in f)

In [450]:
def detect_columns(filename, offset):
    with open(filename, 'r') as f:
        for _ in range(offset):
            next(f)  # skip lines
        line = next(f).strip().split()
        return len(line) - 1  # excluding the trailing "-"

In [451]:
def read_file(filename, rows, cols, offset):
    with open(filename, 'r') as f:
        for _ in range(offset):
            next(f)  # skip lines

        data = []
        for _ in range(rows):
            line = next(f, None)
            if line is None:
                break
            numbers = list(map(int, line.strip().split()[:-1]))  # excluding the trailing "-"
            if len(numbers) != cols:
                break
            data.extend(numbers)

        matrix_rows = len(data) // cols
        return Matrix(data, matrix_rows, cols)

In [452]:
def intersect(row1, row2):
    """Returns True if two rows have a common '1', else False"""
    return any(a == b == 1 for a, b in zip(row1, row2))

def union(row1, row2):
    """Return the union of two rows (bitwise OR)"""
    return [a | b for a, b in zip(row1, row2)]

In [453]:
union([1, 0, 0, 0, 0], [0, 1, 1, 0, 0])

[1, 1, 1, 0, 0]

In [454]:
# def EC(A, B, COV, offset):
#     N = A.rows
#     M = A.cols

#     for i in range(N):
#         # row_data contains the data of the current row
#         # among the N rows of the matrix A
#         # (A is implemented as an array of length N * M)
#         row_data = A.data[i * M:i * M + M] 

#         if sum(row_data) == 0:
#             break

#         if sum(row_data) == M:
#             COV.add((offset + i,))
#             break

#         # Attention: the range of i is [0, N - 1], but the range of j is [0, i + offset]
#         # because A is made of new data only whereas B is increased from the eventual
#         # previous iterations and thus contains more rows than the current A
#         # (if we were to call this algorithm just once with the whole A then
#         # the range of j would be [0, N - 1] but here we might be using a smaller A
#         # of 3 rows whereas B might have 5 rows since we previously worked on the first 2 rows of A)
#         # Since we need the old values of A as well, we are going to read them again
#         # from the file, storing them in a matrix old_A, so that in the following for loop
#         # we are going to use the old values of A (old_A) and the new values of A (A)

# #         for j in range(i + offset):
# #             reading_offset = 0
# #             # j will span from 0 to i+offset excluded and
# #             # when j is less than i, we need to read the old values of A,
# #             # if j is in the first LOADABLE_ROWS rows of A
# #             # then reading_offset will be 0,
# #             # if j is in the second range of LOADABLE_ROWS rows of A
# #             # then reading_offset will be LOADABLE_ROWS,
# #             # so we can compute reading_offset starting from j,
# #             # for example if j is 5 and LOADABLE_ROWS is 3
# #             # then reading_offset will be 3
# #             if j < i:
# #                 # load LOADABLE_ROWS rows from the file
# #                 # starting from reading_offset
# #                 old_A = read_file(FILE_NAME, LOADABLE_ROWS, M, reading_offset)
# #                 reading_offset += LOADABLE_ROWS
# #             # if intersect(A.data[j * M:j * M + M], row_data):
# #                 B[j][i + offset] = 0
# #             else:
# #                 I = {offset + i, j}
# #                 U = union(A.data[j * M:j * M + M], row_data)
# #                 if sum(U) == M:
# #                     COV.add(tuple(sorted(I)))
# #                     B[j][i + offset] = 1
# #                 else:
# #                     B[j][i + offset] = 1
# #                     inter = [k for k in range(j) if B[k][i + offset] and B[k][j]]
# #                     if inter:
# #                         explore(I, U, inter, COV, A, B, offset)

# #     return COV

# def explore(I, U, inter, COV, A, B, offset):
#     N = A.cols
#     for k in inter:
#         i_temp = I.union({k})
#         u_temp = union(U, A.data[k * N:k * N + N])
#         if sum(u_temp) == N:
#             COV.add(tuple(sorted(i_temp)))
#         else:
#             inter_temp = [l for l in inter if l < k and B[l][k]]
#             if inter_temp:
#                 explore(i_temp, u_temp, inter_temp, COV, A, B, offset)



In [455]:
def EC(A, B, COV, offset):
    N = A.rows
    M = A.cols

    comment_lines = count_comment_lines(FILE_NAME)

    for i in range(N):
        row_data = A.data[i * M:i * M + M] 

        if sum(row_data) == 0:
            break

        if sum(row_data) == M:
            COV.add((offset + i,))
            break

        # Iterate over previous rows of A in chunks
        reading_offset = 0
        while reading_offset < i + offset:
            old_A = read_file(FILE_NAME, LOADABLE_ROWS, M, reading_offset+comment_lines)
            for j in range(old_A.rows):
                if intersect(old_A.data[j * M:j * M + M], row_data):
                    B[j + reading_offset][i + offset] = 0
                else:
                    I = {offset + i, j + reading_offset}
                    U = union(old_A.data[j * M:j * M + M], row_data)
                    if sum(U) == M:
                        COV.add(tuple(sorted(I)))
                        B[j + reading_offset][i + offset] = 0
                    else:
                        B[j + reading_offset][i + offset] = 1
                        inter = [k for k in range(j + reading_offset) if B[k][i + offset] and B[k][j + reading_offset]]
                        if inter:
                            explore(I, U, inter, COV, old_A, B, offset)

            reading_offset += LOADABLE_ROWS

    return COV

def explore(I, U, inter, COV, A, B, offset):
    N = A.cols
    for k in inter:
        i_temp = I.union({k})
        u_temp = union(U, A.data[k * N:k * N + N])
        if sum(u_temp) == N:
            COV.add(tuple(sorted(i_temp)))
        else:
            inter_temp = [l for l in inter if l < k and B[l][k]]
            if inter_temp:
                explore(i_temp, u_temp, inter_temp, COV, A, B, offset)

In [456]:
# Testing how set changes when passed to a function and updated
def test_set(s):
    s.update({1, 2, 3, 6, 7})

s = set({1,2,3,4,5})
test_set(s)
s

{1, 2, 3, 4, 5, 6, 7}

In [457]:
def incremental_process(A, B, COV, offset):
    
    # Find exact covers within the new chunk
    chunk_COV = EC(A, B, set(), offset)
    print("Chunk COV:", chunk_COV)
    COV.update(chunk_COV)

    return COV

    # # Process intersections of the new chunk with the previous chunks and update B accordingly
    # for i in range(A.rows):
    #     for j in range(N):
    #         if intersect(A.data[i * A.cols: i * A.cols + A.cols], 
    #                      A.data[j * A.cols: j * A.cols + A.cols]):
    #             B[j][offset + i] = 0
    #         else:
    #             B[j][offset + i] = 1

    # # Process intersections within the new chunk and update B accordingly
    # for i in range(A.rows):
    #     for j in range(i+1, A.rows):
    #         if intersect(A.data[i * A.cols: i * A.cols + A.cols],
    #                      A.data[j * A.cols: j * A.cols + A.cols]):
    #             B[offset + i][offset + j] = 0
    #             B[offset + j][offset + i] = 0
    #         else:
    #             B[offset + i][offset + j] = 1
    #             B[offset + j][offset + i] = 1

    #     if sum(A.data[i * A.cols: i * A.cols + A.cols]) == A.cols:
    #         COV.add((offset + i,))


def main():
    # FILE_NAME = "ec_instance.txt"

    mem_size = 1024 * 1024  # 1 MB for now, can adjust as needed

    comment_lines = count_comment_lines(FILE_NAME)
    print("Comment lines:", comment_lines)

    n_columns = detect_columns(FILE_NAME, comment_lines)
    print(f"Total columns: {n_columns}")


    # Load the initial chunk
    # initial_matrix = read_file(FILE_NAME, LOADABLE_ROWS, n_columns, offset)
    
    total_rows = count_total_lines(FILE_NAME) - comment_lines

    print(f"Total rows: {total_rows}")
    B = [[0] * total_rows for _ in range(total_rows)]
    COV = set()

    # COV = EC(initial_matrix, B, set(), offset-comment_lines) # offset - comment_lines is 0 at the beginning

    # offset += initial_matrix.rows
    # print("Offset after initial chunk:", offset)

    # print("First B found:")
    # for row in B:
    #     print(row)

    # Process subsequent chunks incrementally

    offset = 0

    while True:
        matrix = read_file(FILE_NAME, LOADABLE_ROWS, n_columns, offset+comment_lines)
        if not matrix.data:
            break

        print("Portion of A read: from row", offset+1, "to row", offset+matrix.rows)
        # Remember that matrix.data is an array of integers if length N * M
        # where N is the number of rows and M is the number of columns
        for i in range(matrix.rows):
            print(matrix.data[i * matrix.cols:i * matrix.cols + matrix.cols])


        print("Calling EC")
        incremental_process(matrix, B, COV, offset)

        print("COV:")
        print(COV)
        

        print("B:")
        for row in B:
            print(row)

        offset += matrix.rows

    print("Solutions:")

    # Print solutions
    for solution in COV:
        sets = [f"S_{i+1}" for i in solution]
        print(", ".join(sets))

    print(COV)

if __name__ == "__main__":
    main()

Comment lines: 1
Total columns: 10
Total rows: 8
Portion of A read: from row 1 to row 3
[1, 0, 1, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 1, 1, 1, 0]
[0, 1, 0, 1, 0, 0, 0, 1, 0, 0]
Calling EC
Chunk COV: set()
COV:
set()
B:
[0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
Portion of A read: from row 4 to row 6
[0, 1, 0, 0, 1, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 1, 0, 1, 0]
[0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
Calling EC
Chunk COV: {(0, 1, 3), (0, 2, 4)}
COV:
{(0, 1, 3), (0, 2, 4)}
B:
[0, 1, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
Portion of A read: from row 7 to row 8
[1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 1, 0, 0]
Calling EC
Chunk COV: set()
COV:
{(0, 1, 3), (0, 2, 4)}
B:
[0, 1, 1,